In [1]:
!python3 -m venv clip-env
!source clip-env/bin/activate


In [2]:
!pip install pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

ERROR: Invalid requirement: 'pytorch=1.7.1'
Hint: = is not a valid operator. Did you mean == ?
  Cloning https://github.com/openai/CLIP.git to /private/var/folders/m8/ml9hzx6d3_n565ss8tz38xv40000gn/T/pip-req-build-oq6w38gz
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /private/var/folders/m8/ml9hzx6d3_n565ss8tz38xv40000gn/T/pip-req-build-oq6w38gz
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


In [3]:
pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /private/var/folders/m8/ml9hzx6d3_n565ss8tz38xv40000gn/T/pip-req-build-foqzejgp
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /private/var/folders/m8/ml9hzx6d3_n565ss8tz38xv40000gn/T/pip-req-build-foqzejgp
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade certifi


Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
import clip
from PIL import Image
import requests
from io import BytesIO
import json

# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Function to perform additional feature extraction
def extract_additional_features(image_features):
    # Here, you can perform any additional processing or analysis on the image features
    # For example, you could use clustering, classification, or other techniques
    
    # For demonstration purposes, let's simply calculate the mean of the image features
    additional_features = torch.mean(image_features, dim=1)
    
    return additional_features

def analyze_image(image_url):
    # Retrieve the image from the URL
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))
    
    # Preprocess the image and encode it using CLIP
    img_preprocessed = preprocess(img).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(img_preprocessed)
        
    # Extract additional features from the image features
    additional_features = extract_additional_features(image_features)
    
    # Construct the desired output format
    output = {
        "imageId": image_url.split("/")[-1],  # Extract image ID from URL
        "features": image_features.cpu().numpy().tolist(),
        "additionalFeatures": additional_features.cpu().numpy().tolist(),
        # Add more fields as needed based on your analysis
    }
    
    return output

# Example usage
image_urls = [
    "https://chimabuckets.s3.amazonaws.com/1711275203898_richard-brutyo-Sg3XwuEpybU-unsplash.jpg",
    "https://images.unsplash.com/photo-1710587384959-3541e5e27999?w=800&auto=format&fit=crop&q=60&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxlZGl0b3JpYWwtZmVlZHw0fHx8ZW58MHx8fHx8",
    "https://plus.unsplash.com/premium_photo-1677542200557-3c6856cc98b1?w=800&auto=format&fit=crop&q=60&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxzZWFyY2h8MXx8ZG9nfGVufDB8fDB8fHww"
]
results = [analyze_image(url) for url in image_urls]

print(results)


[{'imageId': '1711275203898_richard-brutyo-Sg3XwuEpybU-unsplash.jpg', 'features': [[0.3105590343475342, 0.13285250961780548, -0.006376619450747967, -0.0160672590136528, 0.29150280356407166, -0.779572069644928, 0.23873458802700043, 0.4919799268245697, -0.5784137845039368, 0.021356895565986633, -0.1568952202796936, -0.48940563201904297, 0.3356722891330719, 0.20349468290805817, 0.7090626358985901, 0.0817461609840393, 0.006493479013442993, -0.0756247267127037, 0.035282447934150696, -0.1190534383058548, 0.02985568717122078, -0.12062966823577881, 0.3543829321861267, 0.2693210542201996, -0.4574240446090698, -0.18364518880844116, 0.25377145409584045, -0.03166739270091057, -0.009002557955682278, 0.012007508426904678, -0.10815772414207458, 0.13533245027065277, -0.2755390405654907, -0.1312735229730606, -0.2113204300403595, 0.3377588093280792, -0.32768306136131287, 0.22800719738006592, -0.5768749713897705, 0.19558104872703552, -0.6919771432876587, 0.2980802655220032, -0.07167713344097137, -0.36544

In [6]:
import torch
import clip
from PIL import Image
import requests
from io import BytesIO
import json

# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Define a set of categories for zero-shot classification
categories = [
    "animal", "vehicle", "outdoor", "food",
    "person", "technology", "landscape", "cityscape"
]

def analyze_image(image_url):
    # Retrieve the image from the URL
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))
    
    # Preprocess the image and encode it using CLIP
    img_preprocessed = preprocess(img).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(img_preprocessed)
        
        # Prepare text tokens for each category
        text_tokens = clip.tokenize(categories).to(device)
        
        # Encode text tokens using CLIP
        text_features = model.encode_text(text_tokens)
        
        # Calculate the similarity between the image and each category
        # Here we use cosine similarity
        similarities = (image_features @ text_features.T).softmax(dim=-1)
        probabilities = similarities.cpu().numpy().flatten()
        
        # Convert numpy float32 to native Python float for JSON serialization
        category_probabilities = {cat: float(prob) for cat, prob in zip(categories, probabilities)}

    # Construct the desired output format
    output = {
        "imageId": image_url.split("/")[-1],  # Extract image ID from URL
        "categories": category_probabilities,
        # Add more fields as needed based on your analysis
    }
    
    return output

# Example usage
image_urls = [
    "https://chimabuckets.s3.amazonaws.com/1711275203898_richard-brutyo-Sg3XwuEpybU-unsplash.jpg",
    "https://images.unsplash.com/photo-1710587384959-3541e5e27999?w=800&auto=format&fit=crop&q=60&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxlZGl0b3JpYWwtZmVlZHw0fHx8ZW58MHx8fHx8",
    "https://plus.unsplash.com/premium_photo-1677542200557-3c6856cc98b1?w=800&auto=format&fit=crop&q=60&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxzZWFyY2h8MXx8ZG9nfGVufDB8fDB8fHww"
]
results = [analyze_image(url) for url in image_urls]

# Print results in JSON format
print(json.dumps(results, indent=2))


[
  {
    "imageId": "1711275203898_richard-brutyo-Sg3XwuEpybU-unsplash.jpg",
    "categories": {
      "animal": 0.1623365432024002,
      "vehicle": 0.00026826473185792565,
      "outdoor": 0.0028488370589911938,
      "food": 0.8195028305053711,
      "person": 0.005402874667197466,
      "technology": 0.009556437842547894,
      "landscape": 7.411785190925002e-05,
      "cityscape": 1.0018714419857133e-05
    }
  },
  {
    "imageId": "photo-1710587384959-3541e5e27999?w=800&auto=format&fit=crop&q=60&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxlZGl0b3JpYWwtZmVlZHw0fHx8ZW58MHx8fHx8",
    "categories": {
      "animal": 0.06899024546146393,
      "vehicle": 0.19771362841129303,
      "outdoor": 0.04725031927227974,
      "food": 0.17010074853897095,
      "person": 0.23135000467300415,
      "technology": 0.2411259263753891,
      "landscape": 0.03853539749979973,
      "cityscape": 0.0049336496740579605
    }
  },
  {
    "imageId": "premium_photo-1677542200557-3c6856cc98b1?w=800&auto=format&

In [7]:
!pip install opencv-python
!pip install numpy
!pip install requests

In [9]:
import cv2
import numpy as np
import os
import requests
from io import BytesIO

def download_image_from_url(image_url):
    """
    Download an image from a URL.
    """
    try:
        response = requests.get(image_url)
        if response.status_code == 200:
            image_data = response.content
            image_array = np.asarray(bytearray(image_data), dtype=np.uint8)
            image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
            return image
        else:
            print("Failed to download image from URL:", image_url)
            return None
    except Exception as e:
        print("Exception occurred while downloading image:", e)
        return None

def preprocess_image(image, target_size=(256, 256)):
    """
    Preprocess an image by resizing and converting it to grayscale.
    """
    if image is None:
        return None
    resized_image = cv2.resize(image, target_size)
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    return gray_image

def extract_color_histogram(image, bins=(8, 8, 8)):
    """
    Extract a 3D color histogram from an image.
    """
    # Convert grayscale image to pseudo-RGB image
    pseudo_rgb_image = cv2.merge([image, image, image])
    hist = cv2.calcHist([pseudo_rgb_image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist


def process_images(image_urls):
    """
    Process images from a list of URLs and extract features.
    """
    features = []
    for image_url in image_urls:
        print("Processing image:", image_url)  # Print the URL being processed
        image = download_image_from_url(image_url)
        if image is None:
            print("Skipping image processing for URL:", image_url)
            continue
        preprocessed_image = preprocess_image(image)
        if preprocessed_image is not None:
            print("Preprocessed image shape:", preprocessed_image.shape)  # Print the shape of the preprocessed image
            print("Preprocessed image data type:", preprocessed_image.dtype)  # Print the data type of the preprocessed image
            hist = extract_color_histogram(preprocessed_image)
            # Store image URL and feature vector in a dictionary
            image_data = {
                "url": image_url,
                "feature_vector": hist.tolist()  # Convert NumPy array to list
            }
            features.append(image_data)
        else:
            print("Failed to preprocess image for URL:", image_url)
    return features

# Example usage:
image_urls = [
    "https://chimabuckets.s3.amazonaws.com/1711275203898_richard-brutyo-Sg3XwuEpybU-unsplash.jpg",
    "https://images.unsplash.com/photo-1710587384959-3541e5e27999?w=800&auto=format&fit=crop&q=60&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxlZGl0b3JpYWwtZmVlZHw0fHx8ZW58MHx8fHx8",
    "https://plus.unsplash.com/premium_photo-1677542200557-3c6856cc98b1?w=800&auto=format&fit=crop&q=60&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxzZWFyY2h8MXx8ZG9nfGVufDB8fDB8fHww"
]

image_features = process_images(image_urls)

print("Number of processed images:", len(image_features))
# Print out the image features
print(image_features)


Processing image: https://chimabuckets.s3.amazonaws.com/1711275203898_richard-brutyo-Sg3XwuEpybU-unsplash.jpg
Preprocessed image shape: (256, 256)
Preprocessed image data type: uint8
Processing image: https://images.unsplash.com/photo-1710587384959-3541e5e27999?w=800&auto=format&fit=crop&q=60&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxlZGl0b3JpYWwtZmVlZHw0fHx8ZW58MHx8fHx8
Preprocessed image shape: (256, 256)
Preprocessed image data type: uint8
Processing image: https://plus.unsplash.com/premium_photo-1677542200557-3c6856cc98b1?w=800&auto=format&fit=crop&q=60&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxzZWFyY2h8MXx8ZG9nfGVufDB8fDB8fHww
Preprocessed image shape: (256, 256)
Preprocessed image data type: uint8
Number of processed images: 3
[{'url': 'https://chimabuckets.s3.amazonaws.com/1711275203898_richard-brutyo-Sg3XwuEpybU-unsplash.jpg', 'feature_vector': [0.19512806832790375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [ ]:

pip install tensorflow
pip install tensorflow-object-detection-api
pip install tensorflow
pip install tf_slim



In [14]:
!pwd
!pip install tensorflow


/Users/gokuljs/work-docs/chima-backend
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.0/227.0 MB 10.8 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 37.1 MB/s eta 0:00:00m eta 0:00:0136m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 36.0 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 37.4 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [16]:
!git clone https://github.com/tensorflow/models.git


Cloning into 'models'...
remote: Enumerating objects: 96720, done.
remote: Total 96720 (delta 0), reused 0 (delta 0), pack-reused 96720
Receiving objects: 100% (96720/96720), 610.81 MiB | 11.51 MiB/s, done.
Resolving deltas: 100% (70412/70412), done.


In [34]:
!cd models/research/object_detection
!protoc 'object_detection/protos/*.proto' --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python3 -m pip install .


Could not make proto path relative: object_detection/protos/*.proto: No such file or directory
cp: object_detection/packages/tf2/setup.py: No such file or directory
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [35]:
import tensorflow as tf
import cv2
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

# Path to the saved_model directory
model_checkpoint_path = '/Users/gokuljs/work-docs/chima-backend/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model'

# Load the model
detect_fn = tf.saved_model.load(model_checkpoint_path)

# Load label map
category_index = label_map_util.create_category_index_from_labelmap(label_map_util.create_categories_from_labelmap())

# Path to the input image
image_path = 'path/to/input/image.jpg'

# Read the input image
image_np = cv2.imread(image_path)

# Convert the image to RGB (TensorFlow Object Detection API expects RGB images)
image_rgb = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)

# Perform object detection
input_tensor = tf.convert_to_tensor(image_rgb)
input_tensor = input_tensor[tf.newaxis, ...]
detections = detect_fn(input_tensor)

# Visualize the detections
viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np,
    detections['detection_boxes'][0].numpy(),
    detections['detection_classes'][0].numpy().astype(int),
    detections['detection_scores'][0].numpy(),
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=200,
    min_score_thresh=0.5,
    agnostic_mode=False)

# Display the image with detections
cv2.imshow('Object Detection', cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'object_detection'